In [3]:
import numpy as np
import healpy as hp
import time

from colossus.cosmology import cosmology
cosmo = cosmology.setCosmology('planck18')
from colossus.halo import profile_nfw
from colossus.halo import concentration
from colossus.halo import mass_so
from scipy import integrate
from scipy.optimize import fsolve
from functools import partial
from astropy import constants as const
from astropy import units as u
from scipy.special import spence
from scipy.optimize import least_squares


In [4]:
# Constant
GG = const.G.to('kpc3 / (M_sun s2)').value # kpc3 / (s2 Msun)
fb = cosmo.Ob0/cosmo.Om0
cc = const.c.to('kpc / s').value


In [5]:
# Functions for rf_gas, rho(r), P(r) 
# (Osato et al 2023: Baryon pasting algorithm: halo-based and particle-based pasting methods)

# Table 1
fstar = 0.026
Sstar = 0.12
Epsilon_star = 3.97e-6
gamma = 1.2
nn = 1./(gamma - 1.)
alpha_nt = 0.18
beta_nt = 0.5
n_nt = 0.8
fmax = 4**(-n_nt)/alpha_nt

def rho_func(rr):
    return np.interp(rr, r_kpc, rho_s) 
def sigv_func(rr):
    return np.interp(rr, r_kpc, sigv_r)
def phi_func(rr):
    return np.interp(rr, r_kpc, phi_s)
def Mr_func(rr):
    return np.interp(rr, r_kpc, M_s)

# eq(10)
def theta_func(rr, rho0, P0):
    return 1. + (gamma-1)/gamma * (rho0/P0) *  (phi0 - phi_func(rr)) 

# eq(8)
def Ptot_func(rr, rho0, P0):
    return P0 * theta_func(rr, rho0, P0)**(nn+1)

# eq(9)
def rhog_func(rr, rho0, P0):
    return rho0 * theta_func(rr, rho0, P0)**(nn)

# eq(15)
def Ef_func(rr, rho0, P0):
    return 4 * np.pi * rr**2 * (1.5 * Ptot_func(rr, rho0, P0) + rhog_func(rr, rho0, P0) * phi_func(rr) )

# ep(12)
def Ei_func(rr):
    dMr = Mr_func(rr+1) - Mr_func(rr)
    return fb * (2 * np.pi * rho_func(rr) * 3 * sigv_func(rr)**2 * rr**2 + phi_func(rr) * dMr)
        
def Mgas_func(rr, rho0, P0):
    return 4 * np.pi * rr**2 * rhog_func(rr, rho0, P0)

def Mgas_int(rr, rho0, P0):
    return integrate.quad(Mgas_func, 0, rr, args=(rho0, P0), limit=300000, epsrel=1e-4)[0] - Mgas

def Mstar_func(rr):
    return 4 * np.pi * rr**2 * rho_func(rr)

def Mstar_int(rr):
    return integrate.quad(Mstar_func, 0, rr, limit=3000, epsrel=1e-3)[0] - Mstar        


In [6]:
edges = np.array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  ])

z_edges = 1./edges - 1.
zbin = (z_edges[1:] + z_edges[:-1])/2.
Nzbin = len(zbin)

Nc_z = np.loadtxt("data/Nc_list.dat", dtype='int')
Mmin_z = np.loadtxt("data/Mmin_list.dat")
Mmax_z = np.loadtxt("data/Mmax_list.dat")


/home/tanimura/anaconda3/envs/pytorch37/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == "":


In [7]:
print(len(edges))
print(len(Nc_z))
print(len(Mmin_z))

101
100
100


In [8]:

Nmbin = 50
rf_list = np.zeros((Nzbin, Nmbin))
rho0_list = np.zeros((Nzbin, Nmbin))
P0_list = np.zeros((Nzbin, Nmbin))

start = time.time()
for I in range(Nzbin):
    
    if(Nc_z[I]==0):
        print(Nc_z[I])
        continue;
        
    mbin = np.logspace(np.log10(Mmin_z[I]),np.log10(Mmax_z[I]), Nmbin)
    
    for J in range(Nmbin):
    #for J in [6]:
        #Mvir = 1E15
        #zz = 0.1
        zz = zbin[I]
        Mvir = mbin[J]
        print("Mvir, zz: ", np.log10(Mvir), zz)
        print("I=%d out of %d, J=%d out of %d" %(I,Nzbin,J,Nmbin))
        middle = time.time()
        print("total time = %f (min)" %( (middle - start)/60.))
        
        Omz = cosmo.Om(zz) 
        vv = 18*np.pi**2 + 82*(Omz-1.) - 39*(Omz-1.)**2
        
        rhoc0 = cosmo.rho_c(0) * cosmo.h**2 # Msun h2/kpc3 -> Msun/kpc3
        rhoc = cosmo.rho_c(zz) * cosmo.h**2 # Msun h2/kpc3 -> Msun/kpc3

        rvir = (3*Mvir/(4*np.pi*vv*rhoc))**(1./3.) # kpc
        cvir = concentration.concentration(Mvir*cosmo.h, 'vir', zz, model = 'ishiyama21')
        r_kpc = 10**np.arange(0,4,0.02) # kpc/h -> kpc

        # Osato et al 2023
        s_vir = r_kpc/rvir # eq(3)
        gc = 1./(np.log(1 + cvir) - cvir /(1 + cvir)) # eq(4)
        cs_vir = cvir*s_vir

        rho_s = rhoc * ( vv * cvir**2 * gc / (3 * s_vir * (1+cs_vir)**2) )
        Mv = 4 * np.pi * (rvir**3) * vv * rhoc / 3.
        M_s = Mv * gc * (np.log(1+cs_vir) - cs_vir/(1+cs_vir))
        Vv = np.sqrt( GG * Mv / rvir)  # km/s -> kpc/s

        phi_s = - Vv**2 * gc * np.log(1 + cs_vir)/s_vir # km2 / s2
        phi0 = - cvir * gc * Vv**2 # km2 / s2

        Li2 = spence(1+cs_vir)
        sigv_r = np.sqrt(Vv**2 * 0.5 * cvir**2 * gc * s_vir * (1. + cs_vir)**2 * ( np.pi**2 - np.log(cs_vir) - 1./cs_vir - 1./(1.+cs_vir)**2  - 6./(1.+cs_vir) + (1. + 1./cs_vir**2 - 4./cs_vir - 2./(1+cs_vir)) * np.log(1.+cs_vir) + 3*(np.log(1+cs_vir))**2 + 6 * Li2) )

        ### Star ###
        Mstar = fstar * (Mvir/3e14)**(-Sstar) * Mvir # Msun
        Estar = Epsilon_star * Mstar * cc**2
        
        ### Gas ###
        Mgas = fb * Mvir - Mstar
        Ps = fb * rho_func(rvir) * sigv_func(rvir)**2
        
        rf_scale = rvir
        p_scale = Ps*1000
        rho_scale = rhoc * 1000

        def Etot_func(params):
            
            rf_fact, rho_fact, P_fact = params
            rf = rf_fact * rf_scale
            rho0 = rho_fact * rho_scale
            P0 = P_fact * p_scale
    
            rstar = fsolve(Mstar_int, x0=100)
            #print('rstar:', rstar)

            delEp = 4.*np.pi*(rvir**3 - rf**3) * Ps / 3.
            Ei = integrate.quad(Ei_func, rstar, rvir)[0]
            Ef = integrate.quad(Ef_func, 0, rf, args=(rho0, P0))[0]
            #print("Ei, Estar, delEp, Ef:",(Ei, Estar, delEp, Ef))
            
            #f1 = ((Estar + delEp + Ef) - Ei)/Ei
            f1 = ((Estar + delEp + Ei) - Ef)/Ei
            f2 = (Ptot_func(rvir, rho0, P0) - Ps)/p_scale
            f3 = (Mgas_int(rf, rho0, P0))/Mgas
            #print("f1, f2, f3: ", f1,f2,f3)
    
            err = np.concatenate(([f1],[f2],[f3]))    
            return err

        try:
            res = least_squares(Etot_func, x0=(1, 1, 1), bounds = ([0, 0, 0], [1000, 1000, 1000]) )
            print("res:", res.x)
            print("rf, rho0, P0: ", res.x * [rf_scale, rho_scale, p_scale])
        
            rf_fit, rho0_fit, P0_fit = res.x * [rf_scale, rho_scale, p_scale]
            rf_list[I][J] = rf_fit
            rho0_list[I][J] = rho0_fit
            P0_list[I][J] = P0_fit
        
        except ValueError as e:
            print('catch ValueError')
            rf_list[I][J] = -1
            rho0_list[I][J] = -1
            P0_list[I][J] = -1
        

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Mvir, zz:  11.632173326852097 3.880952380952381
I=20 out of 100, J=0 out of 50
total time = 0.000033 (min)
res: [2.29578761 0.00689998 0.00475301]
rf, rho0, P0:  [1.14479822e+02 3.23629560e+04 2.40648014e-24]
Mvir, zz:  11.63790448214956 3.880952380952381
I=20 out of 100, J=1 out of 50
total time = 0.391254 (min)
res: [2.28740501 0.00710742 0.004847  ]
rf, rho0, P0:  [1.14564665e+02 3.33359152e+04 2.47371689e-24]
Mvir, zz:  11.643635637447023 3.880952380952381
I=20 out of 100, J=2 out of 50
total time = 0.848368 (min)
res: [2.27806113 0.007328   0.00494193]
rf, rho0, P0:  [1.14599673e+02 3.43705077e+04 2.54571798e-24]
Mvir, zz:  11.649366792744487 3.880952380952381
I=20 out of 100, J=3 out of 50
total time = 1.253250 (min)
res: [2.26872226 0.00755545 0.00503882]
rf, rho0, P0:  [1.14633018e+02 3.54373074e+04 2.62016977e-24]
Mvir, zz:  11.65509794804195 3.880952380952381
I=20 out of 100, J=4 out of 50
total time = 1.707333 (min)
res: [2.25957213 0.

res: [1.94000866 0.02467822 0.0116963 ]
rf, rho0, P0:  [1.14843369e+02 1.15748126e+05 8.33334641e-24]
Mvir, zz:  11.861419538750622 3.880952380952381
I=20 out of 100, J=40 out of 50
total time = 16.437232 (min)
res: [1.93157492 0.02558232 0.01201932]
rf, rho0, P0:  [1.14848202e+02 1.19988629e+05 8.63574587e-24]
Mvir, zz:  11.867150694048087 3.880952380952381
I=20 out of 100, J=41 out of 50
total time = 16.826901 (min)
res: [1.92317877 0.02653464 0.01235913]
rf, rho0, P0:  [1.14853089e+02 1.24455298e+05 8.95319778e-24]
Mvir, zz:  11.87288184934555 3.880952380952381
I=20 out of 100, J=42 out of 50
total time = 17.172567 (min)
res: [1.91488599 0.02750829 0.01270737]
rf, rho0, P0:  [1.14861989e+02 1.29022008e+05 9.27975480e-24]
Mvir, zz:  11.878613004643013 3.880952380952381
I=20 out of 100, J=43 out of 50
total time = 17.555980 (min)
res: [1.90648693 0.02843049 0.0130433 ]
rf, rho0, P0:  [1.14862332e+02 1.33347399e+05 9.60010889e-24]
Mvir, zz:  11.884344159940476 3.880952380952381
I=20 ou

res: [0.75040569 0.92819047 0.27019338]
rf, rho0, P0:  [1.00076431e+02 3.78380809e+06 6.95292835e-22]
Mvir, zz:  12.89599438950475 3.653679653679654
I=21 out of 100, J=29 out of 50
total time = 28.023835 (min)
res: [0.7067223  0.99326186 0.29318811]
rf, rho0, P0:  [9.74565799e+01 4.04907441e+06 8.01613593e-22]
Mvir, zz:  12.939574426147946 3.653679653679654
I=21 out of 100, J=30 out of 50
total time = 28.133898 (min)
res: [0.70357117 1.06409535 0.3116297 ]
rf, rho0, P0:  [1.00322208e+02 4.33783014e+06 9.08109372e-22]
Mvir, zz:  12.98315446279114 3.653679653679654
I=21 out of 100, J=31 out of 50
total time = 28.269145 (min)
res: [0.68700584 1.14154184 0.33442929]
rf, rho0, P0:  [1.01292235e+02 4.65354406e+06 1.03686796e-21]
Mvir, zz:  13.026734499434335 3.653679653679654
I=21 out of 100, J=32 out of 50
total time = 28.434879 (min)
res: [0.65801792 1.23745374 0.36463001]
rf, rho0, P0:  [1.00318292e+02 5.04453305e+06 1.20079431e-21]
Mvir, zz:  13.070314536077529 3.653679653679654
I=21 out

/home/tanimura/anaconda3/envs/pytorch37/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars
/home/tanimura/anaconda3/envs/pytorch37/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars
/home/tanimura/anaconda3/envs/pytorch37/lib/python3.7/site-packages/ipykernel_launcher.py:76: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
/home/tanimura/anaconda3/envs/pytorch37/lib/python3.7/site-packages/ipykernel_launcher.py:49: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.


res: [0.54897331 2.38517821 0.81692278]
rf, rho0, P0:  [1.38227298e+02 9.72328088e+06 6.54938234e-21]
Mvir, zz:  13.724015085725453 3.653679653679654
I=21 out of 100, J=48 out of 50
total time = 30.250768 (min)
res: [0.54913566 2.46091706 0.85456005]
rf, rho0, P0:  [1.42971315e+02 1.00320335e+07 7.26457617e-21]
Mvir, zz:  13.767595122368649 3.653679653679654
I=21 out of 100, J=49 out of 50
total time = 30.310261 (min)
res: [0.53764525 2.58134668 0.91207528]
rf, rho0, P0:  [1.44741064e+02 1.05229700e+07 8.21465994e-21]
Mvir, zz:  11.632173326852097 3.4466403162055337
I=22 out of 100, J=0 out of 50
total time = 30.381173 (min)
res: [2.39159926 0.00539318 0.00415239]
rf, rho0, P0:  [1.30768694e+02 1.92388764e+04 1.43237964e-24]
Mvir, zz:  11.677033674519082 3.4466403162055337
I=22 out of 100, J=1 out of 50
total time = 30.861493 (min)
res: [2.31797053 0.0068082  0.00479988]
rf, rho0, P0:  [1.31182751e+02 2.42866082e+04 1.77730686e-24]
Mvir, zz:  11.721894022186067 3.4466403162055337
I=22 

res: [0.57566531 1.50762039 0.46196503]
rf, rho0, P0:  [1.08720042e+02 5.37807161e+06 1.75015468e-21]
Mvir, zz:  13.292006190530529 3.4466403162055337
I=22 out of 100, J=37 out of 50
total time = 39.259346 (min)
res: [0.60422935 1.58872649 0.47773187]
rf, rho0, P0:  [1.18112215e+02 5.66739803e+06 1.92775667e-21]
Mvir, zz:  13.336866538197514 3.4466403162055337
I=22 out of 100, J=38 out of 50
total time = 39.337623 (min)
res: [0.5878427  1.68998219 0.51262835]
rf, rho0, P0:  [1.18934425e+02 6.02860327e+06 2.19989872e-21]
Mvir, zz:  13.381726885864499 3.4466403162055337
I=22 out of 100, J=39 out of 50
total time = 39.407743 (min)
res: [0.58668903 1.74347177 0.53459009]
rf, rho0, P0:  [1.22859250e+02 6.21941444e+06 2.43619473e-21]
Mvir, zz:  13.426587233531484 3.4466403162055337
I=22 out of 100, J=40 out of 50
total time = 39.509644 (min)
res: [0.58026678 1.81591504 0.56187627]
rf, rho0, P0:  [1.25771157e+02 6.47783829e+06 2.72583384e-21]
Mvir, zz:  13.471447581198468 3.4466403162055337
I

/home/tanimura/anaconda3/envs/pytorch37/lib/python3.7/site-packages/ipykernel_launcher.py:76: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.


res: [0.55253044 2.35634704 0.79942274]
rf, rho0, P0:  [1.52399555e+02 8.40569892e+06 5.90370484e-21]
Mvir, zz:  13.78547001486736 3.4466403162055337
I=22 out of 100, J=48 out of 50
total time = 40.257618 (min)
res: [0.55298339 2.42290498 0.83426143]
rf, rho0, P0:  [1.57867607e+02 8.64312832e+06 6.54559654e-21]
Mvir, zz:  13.830330362534346 3.4466403162055337
I=22 out of 100, J=49 out of 50
total time = 40.375088 (min)
res: [0.54331668 2.54798173 0.8914306 ]
rf, rho0, P0:  [1.60541540e+02 9.08930940e+06 7.42310195e-21]
Mvir, zz:  11.632173326852097 3.2572463768115942
I=23 out of 100, J=0 out of 50
total time = 40.507319 (min)
res: [2.43880554 0.00482589 0.00392672]
rf, rho0, P0:  [1.39199012e+02 1.51587284e+04 1.13022724e-24]
Mvir, zz:  11.679330333226451 3.2572463768115942
I=23 out of 100, J=1 out of 50
total time = 40.945234 (min)
res: [2.36055622 0.00613714 0.00454781]
rf, rho0, P0:  [1.39698689e+02 1.92775166e+04 1.41023914e-24]
Mvir, zz:  11.726487339600807 3.2572463768115942
I=23

KeyboardInterrupt: 

In [12]:
#np.savetxt("data/rf_list.dat", rf_list)
#np.savetxt("data/rho0_list.dat", rho0_list)
#np.savetxt("data/P0_list.dat", P0_list)